In [2]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

Found existing installation: tidyzoning 0.1.0
Uninstalling tidyzoning-0.1.0:
  Successfully uninstalled tidyzoning-0.1.0
  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-7vq_8h1q
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit a6d54bc3941d77afb53c62e1502c5a3ef550ea7e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=4392 sha256=484d550f1c69a212970a47a73e5c7ad3190eb2efc7362344e22cb18c2b860c6d
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-gwwiq0o5\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-7vq_8h1q'


In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

from shapely.geometry import Point
from shapely.geometry import box

# input my libraries
from tidyzoning import find_district_idx
from tidyzoning import find_bldg_type
from tidyzoning import check_land_use

In [5]:
tidybuilding = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tidybuildings/tidybuilding_2family.geojson")
tidyparcel = gpd.read_file(r"C:/Users/Admin/Desktop/zoning_data/Test_Data/Test_Parcels/testparcel_100per_correct.geojson")
tidyzoning = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data//Tidyzoning/tidyzoning_Haltom_City.geojson")

tidyzoning = tidyzoning.to_crs(tidyparcel.crs)
tidybuilding = tidybuilding.to_crs(tidyparcel.crs)

#### The Case Study to use find_district_idx function

In [5]:
find_district_idx_results = find_district_idx(tidyparcel, tidyzoning)
find_district_idx_results

,prop_id,object_id,zoning_id
3,03027805,1000,1.0
8,03027783,1001,1.0
10,03027791,1002,1.0
18,00684619,1003,3.0
22,06687849,1004,3.0
...,...,...,...
75858,03027848,995,1.0
75863,03027856,996,1.0
75868,03027821,997,1.0
75871,04723295,998,1.0


#### The Case Study to use find_bldg_type_results function

In [6]:
find_bldg_type_results = find_bldg_type(tidybuilding)
find_bldg_type_results

'2_family'

#### The Case Study to use check_land_use function

In [8]:
result_df = check_land_use(tidybuilding, tidyzoning)
result_df

NameError: name 'find_bldg_type' is not defined